In [687]:
import pandas as pd
import numpy as np

In [688]:
df=pd.read_csv('hospitalizacion_modelado.csv')
df.head()

,fiebre_SI,itu_SI,tipoCultivo_HEMOCULTIVO,tipoCultivo_HEMOCULTIVO Y UROCULTIVO,tipoCultivo_UROCULTIVO,agenteAislado_E.COLI,agenteAislado_PSEUDOMONAS AERUGINOSA,"patronResistencia_ AMPI R, CIPRO R, GENTA R, SULFA M R",patronResistencia_MULTI SENSIBLE,"patronResistencia_RESISTENTE A AMPI, CIPRO Y GENTA","patronResistencia_RESISTENTE A AMPI, SULFA, CEFADROXILO, CEFUROXIMO, CIPRO Y CEFEPIME, CEFOTAXIMA",hospitalizacion_SI,biopsia_n,fiebre_NO,itu_NO,tipoCultivo_NO,agenteAislado_NO,patronResistencia_NO
0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1
1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1
2,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1
4,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1


In [689]:
# es necesario usar StandardScaler para normalizar el dataset (muestrasTomadas)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#scaled=scaler.fit_transform(np.reshape(df['muestrasTomadas'],(-1,1)))
#raveled=scaled.ravel()
#serie=pd.Series(data=raveled,name='muestrasTomadas')
#df['muestrasTomadas']=serie
#
l = ['biopsia_n']

for i in l:
    scaled=scaler.fit_transform(np.reshape(df[i],(-1,1))).ravel()
    serie=pd.Series(data=scaled,name=i)
    df[i]=serie


In [690]:
df

,fiebre_SI,itu_SI,tipoCultivo_HEMOCULTIVO,tipoCultivo_HEMOCULTIVO Y UROCULTIVO,tipoCultivo_UROCULTIVO,agenteAislado_E.COLI,agenteAislado_PSEUDOMONAS AERUGINOSA,"patronResistencia_ AMPI R, CIPRO R, GENTA R, SULFA M R",patronResistencia_MULTI SENSIBLE,"patronResistencia_RESISTENTE A AMPI, CIPRO Y GENTA","patronResistencia_RESISTENTE A AMPI, SULFA, CEFADROXILO, CEFUROXIMO, CIPRO Y CEFEPIME, CEFOTAXIMA",hospitalizacion_SI,biopsia_n,fiebre_NO,itu_NO,tipoCultivo_NO,agenteAislado_NO,patronResistencia_NO
0,1,0,0,0,0,0,0,0,0,0,0,1,-0.787685,0,1,1,1,1
1,1,0,0,0,0,0,0,0,0,0,0,1,-0.787685,0,1,1,1,1
2,1,0,1,0,0,1,0,1,0,0,0,1,-0.787685,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,1,-0.787685,0,1,1,1,1
4,1,0,1,0,0,1,0,0,0,0,0,1,-0.787685,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,0,0,0,0,0,0,0,0,0,0,0,0,-0.787685,1,1,1,1,1
519,0,0,0,0,0,0,0,0,0,0,0,0,-0.787685,1,1,1,1,1
520,0,0,0,0,0,0,0,0,0,0,0,0,-0.787685,1,1,1,1,1
521,0,0,0,0,0,0,0,0,0,0,0,0,0.832607,1,1,1,1,1


In [691]:
# itero por las columnas y las stackeo
x=[]
for i in df:
    if (i!='hospitalizacion_SI'):
        x.append(np.array(df[i]).reshape(-1,1))

X=np.hstack(x)
y=np.array(df.hospitalizacion_SI).reshape(-1,1)

In [692]:
# ahora hay que realizar el oversampling
# importo imbalanced-learn --> RandomUnderSampler: reduce la mayoria --> RandomOverSampler: amplia la minoria
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='not majority')

X , y = ros.fit_resample(X,y)

# verificar el oversampling
print('value_counts y:',np.unique(y,return_counts=True))
print('len X:',len(X))

value_counts y: (array([0, 1]), array([500, 500]))
len X: 1000


In [693]:
# ahora es necesario realizar una reduccion de la dimensionalidad de este dataset: voy a usar SVD
U, D, V = np.linalg.svd(X)

# voy a quedarme con 15 elementos
e = 5
X = np.matrix(U[:, :e]) * np.diag(D[:e]) * np.matrix(V[:e, :])
X = np.asarray(X)
print(X)

[[ 0.99787601 -0.00466226  0.01522218 ...  0.9906989   0.9906989
   1.00953637]
 [ 0.99787601 -0.00466226  0.01522218 ...  0.9906989   0.9906989
   1.00953637]
 [ 0.97717691  0.05245065  0.90484612 ... -0.06665482 -0.06665482
   0.37639229]
 ...
 [ 1.02817005  1.05334248  0.08870245 ... -0.04881148 -0.04881148
  -0.00569639]
 [ 0.99787601 -0.00466226  0.01522218 ...  0.9906989   0.9906989
   1.00953637]
 [ 0.99787601 -0.00466226  0.01522218 ...  0.9906989   0.9906989
   1.00953637]]


In [694]:
# creo el arbol de decision y pruebo los mejores hiperparametros con GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

arbol = DecisionTreeClassifier()

arbol_params ={'max_depth':np.arange(start=1,stop=11),
               'criterion':['gini','entropy','log_loss'],
               'splitter':['best','random']}

modelo_arbol = GridSearchCV(arbol,param_grid=arbol_params,cv=5)

modelo_arbol.fit(X,y)

Y_Pred = modelo_arbol.predict(X)

In [695]:
modelo_arbol.best_params_

{'criterion': 'gini', 'max_depth': 2, 'splitter': 'best'}

In [696]:
# evaluamos el modelo
from sklearn.metrics import classification_report

print(classification_report(y,Y_Pred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92       500
           1       0.97      0.86      0.91       500

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000



In [697]:
# al parecer el arbol de decision no es el mejor modelo para estos datos, voy a probar un Kneighborsclassifier
from sklearn.neighbors import KNeighborsClassifier

vecinos = KNeighborsClassifier()

vecinos_params ={'n_neighbors':[1,2,3,4,5],
                'weights':['uniform','distance'],
                'algorithm':['auto','ball_tree','kd_tree','brute'],
                'leaf_size':np.arange(start=1,stop=10)}

modelo_vecinos = GridSearchCV(vecinos,param_grid=vecinos_params,cv=5)

modelo_vecinos.fit(X,y)

Y_Pred = modelo_vecinos.predict(X)



In [698]:
modelo_arbol.best_estimator_

DecisionTreeClassifier(max_depth=2)

In [699]:
print(classification_report(y,Y_Pred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92       500
           1       0.97      0.86      0.91       500

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000



In [700]:
modelo_arbol.best_index_

2